<a href="https://colab.research.google.com/github/dhan16/colabs/blob/master/covid19opendata/WikiData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WikiData

In [42]:
# Sparql functions
import requests
import pandas as pd

ENDPOINT = "https://query.wikidata.org/sparql"


def wiki_data(sparql):
  res = requests.get(ENDPOINT, params = {'format': 'json', 'query': sparql})
  return res.json()


def wikidata_to_dataframe(json):
  results = json["results"]["bindings"]
  # column names we draw from the first result
  cols = [ val for val in results[0] ]
  rows = []
  for result in results:
      values = [ result[val]["value"] for val in result ]
      rows.append(values)
  return pd.DataFrame(rows, columns=cols)


In [43]:
# Get all departments of France, and what administrative region they belong to
# SELECT ?place ?subregion1_code ?subregion2_code ?placeLabel
# WHERE
# {
#   ?place wdt:P131 ?subregion1_code.
#   ?place wdt:P2586 ?subregion2_code.
#   ?place wdt:P31/wdt:P279* wd:Q6465.
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
# }

sparql = """
SELECT ?place ?subregion1 ?admin_code ?subregion1Label ?placeLabel ?class ?classLabel
WHERE
{
  ?place wdt:P31/wdt:P279* wd:Q12479774.
  ?place wdt:P31 ?class.
  ?class wdt:P279 wd:Q12479774.
  ?place wdt:P131 ?subregion1.
  ?place wdt:P2588 ?admin_code.
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}
"""
res = wiki_data(sparql)
wiki_raw = wikidata_to_dataframe(res)
wiki_raw
# res

,class,place,admin_code,subregion1,subregion1Label,placeLabel,classLabel
0,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11478,34.03,http://www.wikidata.org/entity/Q3741,Special Region of Yogyakarta,Gunung Kidul,regency of Indonesia
1,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11477,34.02,http://www.wikidata.org/entity/Q3741,Special Region of Yogyakarta,Bantul,regency of Indonesia
2,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11143,35.04,http://www.wikidata.org/entity/Q3586,East Java,Tulungagung,regency of Indonesia
3,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11141,35.23,http://www.wikidata.org/entity/Q3586,East Java,Tuban,regency of Indonesia
4,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q17053892,82.07,http://www.wikidata.org/entity/Q5094,North Maluku,Morotai Island,regency of Indonesia
...,...,...,...,...,...,...,...
536,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8179,19.71,http://www.wikidata.org/entity/Q2271,South Sumatra,Pangkal Pinang,city of Indonesia
537,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8179,19.71,http://www.wikidata.org/entity/Q1866,Bangka Belitung Islands,Pangkal Pinang,city of Indonesia
538,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8158,18.72,http://www.wikidata.org/entity/Q2110,Lampung,Metro,city of Indonesia
539,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8156,18.71,http://www.wikidata.org/entity/Q2271,South Sumatra,Bandar Lampung,city of Indonesia


In [44]:
provs = pd.read_csv('https://raw.githubusercontent.com/dhan16/covid-19-open-data/indo/src/data/metadata.csv')
provs = provs.query('(country_code == "ID")')
provs



,key,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,match_string,aggregate_report_offset
10605,ID,ID,Indonesia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
10606,ID_AC,ID,Indonesia,AC,Aceh,NaN,NaN,NaN,NaN,NaN,0
10607,ID_BA,ID,Indonesia,BA,Bali,NaN,NaN,NaN,NaN,NaN,0
10608,ID_BB,ID,Indonesia,BB,Bangka Belitung Islands,NaN,NaN,NaN,NaN,Kepulauan Bangka Belitung,NaN
10609,ID_BE,ID,Indonesia,BE,Bengkulu,NaN,NaN,NaN,NaN,NaN,NaN
10610,ID_BT,ID,Indonesia,BT,Banten,NaN,NaN,NaN,NaN,NaN,NaN
10611,ID_GO,ID,Indonesia,GO,Gorontalo,NaN,NaN,NaN,NaN,NaN,NaN
10612,ID_JA,ID,Indonesia,JA,Jambi,NaN,NaN,NaN,NaN,NaN,NaN
10613,ID_JB,ID,Indonesia,JB,West Java,NaN,NaN,NaN,NaN,Jawa Barat,NaN
10614,ID_JI,ID,Indonesia,JI,East Java,NaN,NaN,NaN,NaN,Jawa Timur,NaN


In [45]:
wiki_df = wiki_raw.copy()

# wiki_raw.classLabel.unique()
kabkota_to_regiontype = {
    'regency of Indonesia' : 'Regency',
    'administrative regency of Indonesia' : 'Regency',
    'city of Indonesia': 'City',
    'administrative city of Indonesia': 'City',
}
wiki_df['regiontype'] = wiki_df.apply(lambda r: kabkota_to_regiontype[r.classLabel], axis=1)
wiki_df

,class,place,admin_code,subregion1,subregion1Label,placeLabel,classLabel,regiontype
0,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11478,34.03,http://www.wikidata.org/entity/Q3741,Special Region of Yogyakarta,Gunung Kidul,regency of Indonesia,Regency
1,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11477,34.02,http://www.wikidata.org/entity/Q3741,Special Region of Yogyakarta,Bantul,regency of Indonesia,Regency
2,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11143,35.04,http://www.wikidata.org/entity/Q3586,East Java,Tulungagung,regency of Indonesia,Regency
3,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11141,35.23,http://www.wikidata.org/entity/Q3586,East Java,Tuban,regency of Indonesia,Regency
4,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q17053892,82.07,http://www.wikidata.org/entity/Q5094,North Maluku,Morotai Island,regency of Indonesia,Regency
...,...,...,...,...,...,...,...,...
536,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8179,19.71,http://www.wikidata.org/entity/Q2271,South Sumatra,Pangkal Pinang,city of Indonesia,City
537,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8179,19.71,http://www.wikidata.org/entity/Q1866,Bangka Belitung Islands,Pangkal Pinang,city of Indonesia,City
538,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8158,18.72,http://www.wikidata.org/entity/Q2110,Lampung,Metro,city of Indonesia,City
539,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q8156,18.71,http://www.wikidata.org/entity/Q2271,South Sumatra,Bandar Lampung,city of Indonesia,City


## Sheet

In [46]:
!pip install --upgrade gspread

Requirement already up-to-date: gspread in /usr/local/lib/python3.7/dist-packages (3.7.0)


In [47]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

sheet_url = 'https://docs.google.com/spreadsheets/d/1FJJXiGuOb5nXrjJeV3QcHNhTo38YdcsTIFl29mWDIqI/edit#gid=2006070746'
worksheet = gc.open_by_url(sheet_url).worksheet('Kode Kota')
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
sheet_raw = pd.DataFrame.from_records(rows[2:], columns=rows[1])


In [48]:
sheet_df = sheet_raw.copy()

kabkota_to_regiontype = {
    'Kab.' : 'Regency', 
    'Kota': 'City', 
    'zTam' : 'zTam'
}
sheet_df['regiontype'] = sheet_df.apply(lambda r: kabkota_to_regiontype[r.KabKota], axis=1)
sheet_df


,ID Provinsi,Provinsi,ID Kota,KabKota,Kota,regiontype
0,1,Aceh (NAD),258,Kab.,Aceh Barat,Regency
1,1,Aceh (NAD),259,Kab.,Aceh Barat Daya,Regency
2,1,Aceh (NAD),260,Kab.,Aceh Besar,Regency
3,1,Aceh (NAD),261,Kab.,Aceh Jaya,Regency
4,1,Aceh (NAD),262,Kab.,Aceh Selatan,Regency
...,...,...,...,...,...,...
557,34,Sumatera Utara,492,Kab.,Tapanuli Selatan,Regency
558,34,Sumatera Utara,493,Kab.,Tapanuli Tengah,Regency
559,34,Sumatera Utara,494,Kab.,Tapanuli Utara,Regency
560,34,Sumatera Utara,495,Kota,Tebing Tinggi,City


## Sheet vs WikiData

In [49]:
def indonesian_direction_to_english(place: str):
  # place is in lower
  to_english = {
      'pusat': 'central',
      'tengah' : 'central',
      'utara' : 'north',
      'selatan' : 'south',
      'timur' : 'east',
      'barat' : 'west',
  }
  bits = place.split()
  if len(bits) > 1:
    for indo_dir, eng_dir in to_english.items():
      # in indonesian the dir is at the end. island/islands 
      if bits[-1] == indo_dir:
        bits.pop()
        bits.insert(0, eng_dir)
        break
  place = ' '.join(bits)
  return place

def indonesian_to_english(place: str):
  # place is in lower
  to_english = {
      'kepulauan' : 'islands',
      'pulau' : 'island',
  }
  bits = place.split()
  if len(bits) > 1:
    for indo, eng in to_english.items():
      # in indonesian the island is at the beginning while in english its at the end
      if bits[0] == indo:
        bits.pop(0)
        bits.append(eng)
        break
  place = ' '.join(bits)
  return place

# indonesian_direction_to_english('tengah whatever')
indonesian_direction_to_english('whatever tengah')
indonesian_to_english('kepulauan tengah')

'tengah islands'

In [50]:
spellings = {
    'Toba Samosir' : 'Toba Regency',
    'Penajam Paser Utara' : 'Penajam North Paser',
    'Kupang' : 'Kupang Regency',
    'Kepulauan Seribu' : 'Thousand Islands',
    'Pangkajene Kepulauan' : 'Pangkajene Islands',
    'Pasangkayu (Mamuju Utara)': 'Pasangkayu', # https://en.wikipedia.org/wiki/Pasangkayu_Regency
    # due to dashes
    'Bau-Bau' : 'Baubau',
    'Tojo Una-Una' : 'Tojo Una Una',
    'Toli-Toli': 'Tolitoli',
     # verify below
    'Kepulauan Sangihe' : 'Sangihe',
    'Kepulauan Sitaro' : 'Kepulauan Siau Tagulandang Biaro',
}
def standard_place_spelling(place: str):
   # other spellings if still required
  place = spellings.get(place, place)
  # all lowercase
  place = place.lower()
  # apply translations
  place = indonesian_direction_to_english(place)
  place = indonesian_to_english(place)
  # remove spaces
  place = place.replace(' ', '')
  return place


wiki_df['place_standardised'] = wiki_df.apply(lambda r: standard_place_spelling(r.placeLabel), axis=1)
sheet_df['Kota_standardised'] = sheet_df.apply(lambda r: standard_place_spelling(r.Kota), axis=1)


In [51]:
# Find rows in wiki_df with no matches in sheet_df
df = wiki_df.merge(sheet_df, how='left', right_on=['Kota_standardised', 'regiontype'], left_on=['place_standardised', 'regiontype']) 
df = df.merge(provs, how='left', left_on='subregion1Label', right_on='subregion1_name')
missing = df[df['Kota'].isnull()]
missing
# len(missing)

df = df[~df['Kota'].isnull()]
cols = ['country_code', 'country_name', 'subregion1_code', 'subregion1_name', 'admin_code', 'placeLabel']
df[cols]

,country_code,country_name,subregion1_code,subregion1_name,admin_code,placeLabel
0,NaN,NaN,NaN,NaN,34.03,Gunung Kidul
1,NaN,NaN,NaN,NaN,34.02,Bantul
2,ID,Indonesia,JI,East Java,35.04,Tulungagung
3,ID,Indonesia,JI,East Java,35.23,Tuban
4,ID,Indonesia,MU,North Maluku,82.07,Morotai Island
...,...,...,...,...,...,...
536,ID,Indonesia,SS,South Sumatra,19.71,Pangkal Pinang
537,ID,Indonesia,BB,Bangka Belitung Islands,19.71,Pangkal Pinang
538,ID,Indonesia,LA,Lampung,18.72,Metro
539,ID,Indonesia,SS,South Sumatra,18.71,Bandar Lampung


In [52]:
# Find rows in sheet_df with no matches in wiki_df
df = sheet_df.merge(wiki_df, how='left', left_on=['Kota_standardised', 'regiontype'], right_on=['place_standardised', 'regiontype']) 
# missing = df[df['placeLabel'].isnull() & ~df['KabKota'].isin(['zTam'])]
missing = df[df['placeLabel'].isnull()]
missing
# len(missing)

,ID Provinsi,Provinsi,ID Kota,KabKota,Kota,regiontype,Kota_standardised,class,place,admin_code,subregion1,subregion1Label,placeLabel,classLabel,place_standardised
23,1,Aceh (NAD),1011,zTam,Luar Provinsi Aceh,zTam,luarprovinsiaceh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,1,Aceh (NAD),1013,zTam,Luar Negeri (Aceh),zTam,luarnegeri(aceh),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,1,Aceh (NAD),1012,zTam,Belum Diverifikasi (Aceh),zTam,belumdiverifikasi(aceh),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2,Bali,1021,zTam,WNA (Bali),zTam,wna(bali),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2,Bali,1022,zTam,Luar Provinsi Bali,zTam,luarprovinsibali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,2,Bali,1023,zTam,Belum Diverifikasi (Bali),zTam,belumdiverifikasi(bali),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,3,Banten,21,Kota,Serang,City,serang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,3,Banten,1031,zTam,Belum Diverifikasi (Banten),zTam,belumdiverifikasi(banten),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,4,Bengkulu,1041,zTam,Belum Diverifikasi (Bengkulu),zTam,belumdiverifikasi(bengkulu),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,5,DI Yogyakarta,1051,zTam,Luar Provinsi DIY,zTam,luarprovinsidiy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
